Imports needed for the script

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### File Input Handling

In [20]:
#Files for the data to import
Acc_files = ['Data/Acc_Circle_1.csv', 'Data/Acc_Circle_2.csv', 'Data/Acc_Circle_3.csv', 'Data/Acc_Stand_1.csv',
            'Data/Acc_Stand_2.csv','Data/Acc_Stand_3.csv', 'Data/Acc_Down_1.csv', 'Data/Acc_Down_2.csv', 'Data/Acc_Down_3.csv']
Gscope_files = ['Data/Gscope_Circle_1.csv', 'Data/Gscope_Circle_2.csv', 'Data/Gscope_Circle_3.csv', 'Data/Gscope_Stand_1.csv',
            'Data/Gscope_Stand_2.csv','Data/Gscope_Stand_3.csv', 'Data/Gscope_Down_1.csv', 'Data/Gscope_Down_2.csv', 'Data/Gscope_Down_3.csv']

#Empty list to append the data into
Acc_dataframes = []
Gscope_dataframes = []

#Loop over every file and add it to the list for Accelerometer and Gscope
for file in Acc_files:
    Acc_dataframes.append(pd.read_csv(file, usecols=[1,2,3], names=['ax', 'ay', 'az'], skiprows=[0]))

for file in Gscope_files:
    Gscope_dataframes.append(pd.read_csv(file, usecols=[1,2,3], names=['gx', 'gy', 'gz'], skiprows=[0]))

### $\color{LightBlue}{\text{Plotting the Accelerometer Data}}$

In [ ]:
# Iterate over every file and plot the graph for it
for i, acc_data in enumerate(Acc_dataframes):
    filename = os.path.basename(Acc_files[i]) # Get the file name
    measurement_type = filename.split('_')[1]
    title = f"Accelerometer Data - {measurement_type} Recording {i % 3 + 1}" # Fix title for the graph
    acc_data.plot(subplots=True, title=title, xlabel="Samples", ylabel="Acceleration (g)")
    plt.show()

### $\color{LightBlue}{\text{Plotting The Gyroscope Data}}$

In [ ]:
# Iterate over every file and plot the graph for it
for i, gscope_data in enumerate(Gscope_dataframes):
    filename = os.path.basename(Gscope_files[i]) # Get the file name
    measurement_type = filename.split('_')[1]
    title = f"Gyroscope Data - {measurement_type} Recording {i % 3 + 1}" # Fix title for the graph
    gscope_data.plot(subplots=True, title=title, xlabel="Samples", ylabel="Rotation")
    plt.show()

# Questions to Answer:

### $\color{LightBlue}{\text{What are we measuring with the Accelerometer?}}$
The accelerometer measures the vibration and motion of a structure.

### $\color{LightBlue}{\text{What are we measuring the with Gyroscope?}}$
The gyroscope measures rotational motion.

### $\color{LightBlue}{\text{By looking at the plots can you see the difference between the the three classes?}}$

### Accelerometer:
Moving in a Circle: The phone was layed flat on the table and rotated around the Z-axis, the accelerometer detected movement on the X and Y axes but not on the Z since there is no movement on the Z axis.

Standing Up: The phone was placed vertically standing up. The accelerometer didn't detect much movement on the X and Z axes, but movement on the Y-Axis was detected.

Laying Down: The phone was layed down on its back on the table, small movement was detected due to small vibrations and movements around the phone.


### Gyroscope:
Moving in a Circle: The experiment was the same as the accelerometer. The gyroscope detected most of the rotations on the Z-axis but not much on the X and Y axes.

Standing Up: No rotation on all axes.

Laying Down: No rotation on all axes.


In [ ]:
def mergeFiles():
    pass

def saveToTraining():
    pass

def saveToTest():
    pass

def loadFromTraining():
    pass

def loadFromTest():
    pass

